In [3]:
from sklearn.utils.extmath import softmax
import numpy as np

In [5]:
x = np.array([[0.5, 0]])
softmax(x)

array([[0.62245933, 0.37754067]])

In [6]:
x = np.array([[0.5, 0.5]])
softmax(x)

array([[0.5, 0.5]])

In [11]:
x = np.array([[0, 0.99]])
softmax(x)

array([[0.27091208, 0.72908792]])